In [19]:
from google.cloud import bigquery
import pandas as pd
from datetime import date,datetime,timedelta,timezone
from google.cloud import bigquery
from google.oauth2 import service_account
from dotenv import dotenv_values
from datetime import datetime ,timezone

In [20]:
isLoadingAll=True
model_version='model_v2_t150723'

env_path=r'D:\PythonDev\MyQuantFinProject\SMart-AI\.env'
config = dotenv_values(dotenv_path=env_path)

In [21]:
projectId='smart-data-ml'
dataset_id='SMartML'
table_id = f"{projectId}.{dataset_id}.new_result_prediction_incident"

credentials = service_account.Credentials.from_service_account_file(r'C:\Windows\smart-data-ml-91b6f6204773.json')
client = bigquery.Client(credentials= credentials,project=projectId)
# client = bigquery.Client(project=projectId)

In [22]:
today=datetime.strptime(datetime.now(timezone.utc).strftime("%Y-%m-%d"),"%Y-%m-%d")
predict_datetime = today - timedelta(days = 1) 
#predict_datetime = date.today()
print(f"Today {today} : Get prediction as of {predict_datetime}")

Today 2023-07-20 00:00:00 : Get prediction as of 2023-07-19 00:00:00


In [23]:

def load_data_bq(sql:str):

 query_result=client.query(sql)
 df_all=query_result.to_dataframe()
 return df_all

if isLoadingAll:
    sql=f""" select id,predict_severity,prediction_datetime from {table_id} """
else:
    sql=f"""
    select id,predict_severity,prediction_datetime from {table_id} 
     where Date(prediction_datetime)='{predict_datetime}' """
    
print(sql)

df=load_data_bq(sql)
if len(df)>0:
    df.columns=['incident_id','severity_label','prediction_at']
else:
    quit()

 select id,predict_severity,prediction_datetime from smart-data-ml.SMartML.new_result_prediction_incident 


In [24]:
# load json file
map_severity_to_class={'Cosmetic':0,'Minor': 1, "Major": 2, "Critical": 3}
revert_class_to_severity = {v: k for k, v in map_severity_to_class.items()}

print(f"Map severity Name to LabelCode: {str(revert_class_to_severity)}")

df['severity_name']=df['severity_label'].map(revert_class_to_severity)

update_at=datetime.now()
df['imported_at']=update_at
df['model_version']=model_version

Map severity Name to LabelCode: {0: 'Cosmetic', 1: 'Minor', 2: 'Major', 3: 'Critical'}


In [25]:
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   incident_id     37 non-null     Int64         
 1   severity_label  37 non-null     Int64         
 2   prediction_at   37 non-null     datetime64[ns]
 3   severity_name   37 non-null     object        
 4   imported_at     37 non-null     datetime64[ns]
 5   model_version   37 non-null     object        
dtypes: Int64(2), datetime64[ns](2), object(2)
memory usage: 1.9+ KB
None


,incident_id,severity_label,prediction_at,severity_name,imported_at,model_version
0,2409,0,2023-07-20 12:07:49.329440,Cosmetic,2023-07-20 20:46:46.320712,model_v2_t150723
1,3172,0,2023-07-20 12:07:49.329440,Cosmetic,2023-07-20 20:46:46.320712,model_v2_t150723
2,3191,0,2023-07-20 12:07:49.329440,Cosmetic,2023-07-20 20:46:46.320712,model_v2_t150723
3,3190,0,2023-07-20 12:07:49.329440,Cosmetic,2023-07-20 20:46:46.320712,model_v2_t150723
4,3189,0,2023-07-20 12:07:49.329440,Cosmetic,2023-07-20 20:46:46.320712,model_v2_t150723
5,2966,1,2023-07-20 12:07:49.329440,Minor,2023-07-20 20:46:46.320712,model_v2_t150723
6,3192,1,2023-07-20 12:07:49.329440,Minor,2023-07-20 20:46:46.320712,model_v2_t150723
7,3173,1,2023-07-20 12:07:49.329440,Minor,2023-07-20 20:46:46.320712,model_v2_t150723
8,3196,1,2023-07-20 12:07:49.329440,Minor,2023-07-20 20:46:46.320712,model_v2_t150723
9,3169,1,2023-07-20 12:07:49.329440,Minor,2023-07-20 20:46:46.320712,model_v2_t150723


In [26]:
import psycopg2
import psycopg2.extras as extras

def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error

def add_data_values(df, table,conn):

    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    #print(query)
    #return query,tuples
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        raise error
        return 0
    
    return 1
    cursor.close()
    
result=add_data_values(df,'app_prediction_ml_severity_incident',get_postgres_conn())

if  result==1:
    print(f"{len(df.index)} items have been imported to database successfully.")
    print("importing data succeeded")

37 items have been imported to database successfully.
importing data succeeded
